In [36]:
def as_list(x):
    if type(x) is list:
        return x
    else:
        return [x]

In [37]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np

# global variables
api_key = 'RGAPI-835f0471-6022-4dfc-9b5c-ce2c6271ad67'
watcher = LolWatcher(api_key)
my_region = 'euw1'


In [38]:
# fetch matchlist by puuid
me = watcher.summoner.by_name(my_region, 'Kelody')
# me = watcher.summoner.by_name(my_region, 'daimox')
my_matches = watcher.match.matchlist_by_puuid("EUROPE", me['puuid'])
matches = []
matchTimeLine = []
participants = []


In [39]:
# fetch game, participant and match timeline and transform into dataframe
for matchId in my_matches:
    match_detail = watcher.match.by_id("EUROPE", matchId)
    timeline = watcher.match.timeline_by_match("EUROPE", matchId)
    if match_detail["info"]["mapId"] in [1, 2, 11] and match_detail["info"]["gameMode"] == "CLASSIC":
        match = {}
        match["matchId"] = matchId
        match["gameDuration"] = match_detail["info"]["gameDuration"]
        match["gameType"] = match_detail["info"]["gameType"]
        match["gameVersion"] = match_detail["info"]["gameVersion"]
        matches.append(match)
        for index, row in enumerate(match_detail["info"]["participants"]):
            participants_row = {}
            participants_row["matchId"] = matchId
            participants_row["participant"] = index + 1
            participants_row["summonerName"] = row["summonerName"]
            participants_row["teamPosition"] = row["teamPosition"]
            participants_row["championName"] = row["championName"]
            participants_row["kda"] = f"{row['kills']}/{row['deaths']}/{row['assists']}"
            participants_row["champLevel"] = row["champLevel"]
            participants_row["totalDamageToChampions"] = row[
                "totalDamageDealtToChampions"
            ]
            participants_row["totalDamageTaken"] = row["totalDamageTaken"]
            participants_row["totalHeal"] = row["totalHeal"]
            participants_row["totalMinionsKilled"] = row["totalMinionsKilled"]
            participants_row["goldEarned"] = row["goldEarned"]
            participants_row["summoner1"] = row["summoner1Id"]
            participants_row["summoner2"] = row["summoner2Id"]
            participants_row["item0"] = row["item0"]
            participants_row["item1"] = row["item1"]
            participants_row["item2"] = row["item2"]
            participants_row["item3"] = row["item3"]
            participants_row["item4"] = row["item4"]
            participants_row["item5"] = row["item5"]
            participants_row["item6"] = row["item6"]
            participants_row["win"] = row["win"]
            participants.append(participants_row)

        for row in timeline["info"]["frames"]:
            # frameCounter = 1
            # frame = row["{:02d}".format(frameCounter)]
            # frameCounter += 1
            for frameRow in row["events"]:
                timeline_row = {}
                timeline_row["matchId"] = matchId
                eventCounter = 0
                # eventFrame = frameRow["{:02d}".format(eventCounter)]
                eventFrame = frameRow
                eventCounter += 1

                if eventFrame["type"] == "CHAMPION_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["victimId"] = eventFrame["victimId"]
                    if "assistingParticipantIds" in eventFrame:
                        timeline_row["assistingParticipantIds"] = eventFrame[
                            "assistingParticipantIds"
                        ]
                    if "bounty" in eventFrame:
                        timeline_row["bounty"] = eventFrame["bounty"]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "BUILDING_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["teamId"] = eventFrame["teamId"]
                    timeline_row["buildingType"] = eventFrame["buildingType"]
                    timeline_row["laneType"] = eventFrame["laneType"]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "ITEM_PURCHASED":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["participantId"] = eventFrame["participantId"]
                    timeline_row["itemId"] = eventFrame["itemId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "WARD_PLACED":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["wardType"] = eventFrame["wardType"]
                    timeline_row["creatorId"] = eventFrame["creatorId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "WARD_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["wardType"] = eventFrame["wardType"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "LEVEL_UP":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["participantId"] = eventFrame["participantId"]
                    timeline_row["level"] = eventFrame["level"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "ELITE_MONSTER_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["teamId"] = eventFrame["killerTeamId"]
                    if "assistingParticipantIds" in eventFrame:
                        timeline_row["assistingParticipantIds"] = eventFrame[
                            "assistingParticipantIds"
                        ]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    timeline_row["monsterType"] = eventFrame["monsterType"]
                    matchTimeLine.append(timeline_row)


participantsDf = pd.DataFrame(participants)
matchesDf = pd.DataFrame(matches)
matchTimelineDf = pd.DataFrame(matchTimeLine)
matchTimelineDf["timestampMIN"] = round(matchTimelineDf["timestamp"] / 60000, 0)


In [40]:
participantsDf

,matchId,participant,summonerName,teamPosition,championName,kda,champLevel,totalDamageToChampions,totalDamageTaken,totalHeal,...,summoner1,summoner2,item0,item1,item2,item3,item4,item5,item6,win
0,EUW1_5829524931,1,Tanba,TOP,Tryndamere,11/9/3,17,26884,37297,12644,...,4,6,3006,6675,1038,3031,3033,6672,3340,True
1,EUW1_5829524931,2,stoeckchen96,JUNGLE,Kayn,6/2/16,16,14047,26007,14146,...,4,11,6333,6630,3133,3047,3053,3067,3364,True
2,EUW1_5829524931,3,Christopheres,MIDDLE,Katarina,13/8/7,15,21707,29476,8327,...,14,4,2421,3020,6632,6333,3053,1055,3340,True
3,EUW1_5829524931,4,iceblooded,BOTTOM,Aphelios,11/10/13,16,19910,24417,6863,...,4,7,6676,6671,3156,3006,3031,1031,3363,True
4,EUW1_5829524931,5,PlzClickLantern,UTILITY,Thresh,6/7/24,15,12296,21937,3786,...,4,14,3190,4643,3860,3050,3117,3109,3364,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,EUW1_5825747550,6,From Unit 731,TOP,Camille,8/12/2,13,22097,29258,4006,...,12,14,6333,6632,3123,3074,1001,1036,3340,False
176,EUW1_5825747550,7,Treul,JUNGLE,FiddleSticks,2/8/10,12,10109,21923,7659,...,4,11,4636,1058,3020,1082,0,0,3330,False
177,EUW1_5825747550,8,Helium Submarine,MIDDLE,Malzahar,1/13/4,13,10344,19379,269,...,4,12,1056,6653,3116,3020,0,0,3340,False
178,EUW1_5825747550,9,K3nSter,BOTTOM,MissFortune,6/10/6,13,15574,16774,719,...,7,4,6692,3156,3004,3009,0,0,3363,False


In [41]:
matchTimelineDf["assistingParticipantIds"] = [ [] if x is np.NaN else x for x in matchTimelineDf["assistingParticipantIds"]]

In [42]:
def getSummonerName(row, type):
    matchId = row["matchId"]
    if type == "participant":
        if pd.notna(row["participantId"]):
            participantId = row["participantId"]
            #print(participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values)
            return participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
        else:
            return float('NaN')
    elif type == "killer":
        if pd.notna(row["killerId"]):
            killerId = int(row["killerId"])
            try:
                return participantsDf.loc[(participantsDf["participant"] == killerId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "victim":
        if pd.notna(row["victimId"]):
            victimId = int(row["victimId"])
            try:
                return participantsDf.loc[(participantsDf["participant"] == victimId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "assist":
        pass
        #todo

def getChampionName(row, type):
    matchId = row["matchId"]
    if type == "participant":
        if pd.notna(row["participantId"]):
            participantId = row["participantId"]
            #print(participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values)
            return participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
        else:
            return float('NaN')
    elif type == "killer":
        if pd.notna(row["killerId"]):
            killerId = row["killerId"]
            try:
                return participantsDf.loc[(participantsDf["participant"] == killerId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "victim":
        if pd.notna(row["victimId"]):
            victimId = row["victimId"]
            try:
                return participantsDf.loc[(participantsDf["participant"] == victimId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "assist":
        pass
        #todo

def checkAssist(row, name):
    matchId = row["matchId"]
    if row["assistingParticipantIds"]:
        assistingParticipants = row["assistingParticipantIds"]
        for i in range(len(assistingParticipants)):
            if participantsDf.loc[(participantsDf["participant"] == assistingParticipants[i]) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0] == name:
                return True
    return False


In [43]:
matchTimelineDf["participantName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "participant"), axis=1)
matchTimelineDf["participantChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "participant"), axis=1)
matchTimelineDf["killerName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "killer"), axis=1)
matchTimelineDf["killerChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "killer"), axis=1)
matchTimelineDf["victimName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "victim"), axis=1)
matchTimelineDf["victimChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "victim"), axis=1)
matchTimelineDf["kelodyAssist"] = matchTimelineDf.apply(lambda row: checkAssist(row, "Kelody"), axis = 1)


In [44]:
matchTimelineDf

,matchId,type,participantId,itemId,timestamp,level,wardType,creatorId,killerId,victimId,...,buildingType,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist
0,EUW1_5829524931,ITEM_PURCHASED,6.0,2033.0,3348,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,InRevolt,Jax,NaN,NaN,NaN,NaN,False
1,EUW1_5829524931,ITEM_PURCHASED,6.0,3340.0,4141,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,InRevolt,Jax,NaN,NaN,NaN,NaN,False
2,EUW1_5829524931,ITEM_PURCHASED,3.0,1055.0,6122,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Christopheres,Katarina,NaN,NaN,NaN,NaN,False
3,EUW1_5829524931,ITEM_PURCHASED,3.0,2003.0,6122,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Christopheres,Katarina,NaN,NaN,NaN,NaN,False
4,EUW1_5829524931,ITEM_PURCHASED,10.0,3854.0,6387,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Fegel,Pyke,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12092,EUW1_5825747550,ITEM_PURCHASED,6.0,6333.0,1496001,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,25.0,From Unit 731,Camille,NaN,NaN,NaN,NaN,False
12093,EUW1_5825747550,CHAMPION_KILL,NaN,NaN,1497158,NaN,NaN,NaN,5.0,10.0,...,NaN,NaN,25.0,NaN,NaN,Fegel,Nautilus,Big Moan,Teemo,False
12094,EUW1_5825747550,ITEM_PURCHASED,6.0,1036.0,1497291,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,25.0,From Unit 731,Camille,NaN,NaN,NaN,NaN,False
12095,EUW1_5825747550,BUILDING_KILL,NaN,NaN,1500438,NaN,NaN,NaN,0.0,NaN,...,TOWER_BUILDING,TOP_LANE,25.0,NaN,NaN,NaN,NaN,NaN,NaN,False


In [45]:
kelody = participantsDf[participantsDf["summonerName"] == "Kelody"][["matchId", "participant"]]
kelody

,matchId,participant
7,EUW1_5829524931,8
12,EUW1_5829408711,3
27,EUW1_5829375821,8
36,EUW1_5827639869,7
46,EUW1_5827587343,7
58,EUW1_5827621876,9
67,EUW1_5827066877,8
77,EUW1_5827020811,8
88,EUW1_5826907936,9
93,EUW1_5826932804,4


In [46]:
kelodyEvents = []
#TODO bool if kelody was assist or not
# (matchTimelineDf["assistingParticipantIds"] == kelody["participant"].values[0]
for index, match in kelody.iterrows():
    game = []
    assist = matchTimelineDf[(matchTimelineDf["matchId"] == match["matchId"]) & matchTimelineDf["assistingParticipantIds"].notna() & (matchTimelineDf["creatorId"] != match["participant"]) & (matchTimelineDf["killerId"] != match["participant"]) & (matchTimelineDf["victimId"] != match["participant"])]
    participantList = as_list(match["participant"])
    game = matchTimelineDf.loc[((matchTimelineDf["matchId"] == match["matchId"]) & ((matchTimelineDf["participantId"] == (match["participant"])) | 
    (matchTimelineDf["creatorId"] == match["participant"]) |
    (matchTimelineDf["killerId"] == match["participant"]) |
    (matchTimelineDf["victimId"] == match["participant"]) |
    (assist["assistingParticipantIds"].apply(lambda x: any(i in x for i in participantList)))))]
    kelodyEvents.append(game)
kelodyEventsDf = pd.concat(kelodyEvents, ignore_index=True)
kelodyEventsDf

,matchId,type,participantId,itemId,timestamp,level,wardType,creatorId,killerId,victimId,...,buildingType,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist
0,EUW1_5829524931,ITEM_PURCHASED,8.0,1056.0,30738,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,Kelody,Diana,NaN,NaN,NaN,NaN,False
1,EUW1_5829524931,ITEM_PURCHASED,8.0,2003.0,30738,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,Kelody,Diana,NaN,NaN,NaN,NaN,False
2,EUW1_5829524931,ITEM_PURCHASED,8.0,2003.0,30738,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,Kelody,Diana,NaN,NaN,NaN,NaN,False
3,EUW1_5829524931,LEVEL_UP,8.0,NaN,125948,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,2.0,Kelody,Diana,NaN,NaN,NaN,NaN,False
4,EUW1_5829524931,LEVEL_UP,8.0,NaN,155811,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,Kelody,Diana,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319,EUW1_5825747550,CHAMPION_KILL,NaN,NaN,1377765,NaN,NaN,NaN,9.0,3.0,...,NaN,NaN,23.0,NaN,NaN,K3nSter,MissFortune,Kelody,Kassadin,False
1320,EUW1_5825747550,ITEM_PURCHASED,3.0,1058.0,1378987,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,23.0,Kelody,Kassadin,NaN,NaN,NaN,NaN,False
1321,EUW1_5825747550,CHAMPION_KILL,NaN,NaN,1468182,NaN,NaN,NaN,3.0,9.0,...,NaN,NaN,24.0,NaN,NaN,Kelody,Kassadin,K3nSter,MissFortune,False
1322,EUW1_5825747550,CHAMPION_KILL,NaN,NaN,1478422,NaN,NaN,NaN,6.0,3.0,...,NaN,NaN,25.0,NaN,NaN,From Unit 731,Camille,Kelody,Kassadin,False


In [47]:
kelodyEventsDf[kelodyEventsDf["kelodyAssist"] == True]

,matchId,type,participantId,itemId,timestamp,level,wardType,creatorId,killerId,victimId,...,buildingType,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist
10,EUW1_5829524931,CHAMPION_KILL,NaN,NaN,254341,NaN,NaN,NaN,7.0,3.0,...,NaN,NaN,4.0,NaN,NaN,Zamboor,Qiyana,Christopheres,Katarina,True
26,EUW1_5829524931,CHAMPION_KILL,NaN,NaN,530162,NaN,NaN,NaN,7.0,3.0,...,NaN,NaN,9.0,NaN,NaN,Zamboor,Qiyana,Christopheres,Katarina,True
32,EUW1_5829524931,CHAMPION_KILL,NaN,NaN,664530,NaN,NaN,NaN,7.0,4.0,...,NaN,NaN,11.0,NaN,NaN,Zamboor,Qiyana,iceblooded,Aphelios,True
33,EUW1_5829524931,CHAMPION_KILL,NaN,NaN,670641,NaN,NaN,NaN,7.0,5.0,...,NaN,NaN,11.0,NaN,NaN,Zamboor,Qiyana,PlzClickLantern,Thresh,True
39,EUW1_5829524931,CHAMPION_KILL,NaN,NaN,768979,NaN,NaN,NaN,7.0,5.0,...,NaN,NaN,13.0,NaN,NaN,Zamboor,Qiyana,PlzClickLantern,Thresh,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303,EUW1_5825747550,CHAMPION_KILL,NaN,NaN,1101926,NaN,NaN,NaN,2.0,10.0,...,NaN,NaN,18.0,NaN,NaN,Krito2001,Vi,Big Moan,Teemo,True
1305,EUW1_5825747550,CHAMPION_KILL,NaN,NaN,1105131,NaN,NaN,NaN,1.0,6.0,...,NaN,NaN,18.0,NaN,NaN,InRevolt,Aatrox,From Unit 731,Camille,True
1307,EUW1_5825747550,CHAMPION_KILL,NaN,NaN,1161841,NaN,NaN,NaN,5.0,8.0,...,NaN,NaN,19.0,NaN,NaN,Fegel,Nautilus,Helium Submarine,Malzahar,True
1312,EUW1_5825747550,CHAMPION_KILL,NaN,NaN,1261228,NaN,NaN,NaN,4.0,6.0,...,NaN,NaN,21.0,NaN,NaN,Sir Nurum,Ezreal,From Unit 731,Camille,True


Check how often a Player died in Early Game (pre 10 mins)

In [48]:
sumOfDeaths = []
for index, match in kelody.iterrows():
     deaths = len(kelodyEventsDf[(kelodyEventsDf["matchId"] == match["matchId"]) & (kelodyEventsDf["victimId"] == match["participant"]) & (kelodyEventsDf["timestamp"] <= 600000)].index)
     sumOfDeaths.insert(len(sumOfDeaths), deaths)
kelody["deathsPreMin10"] = sumOfDeaths
kelody

,matchId,participant,deathsPreMin10
7,EUW1_5829524931,8,1
12,EUW1_5829408711,3,1
27,EUW1_5829375821,8,0
36,EUW1_5827639869,7,2
46,EUW1_5827587343,7,2
58,EUW1_5827621876,9,3
67,EUW1_5827066877,8,4
77,EUW1_5827020811,8,2
88,EUW1_5826907936,9,4
93,EUW1_5826932804,4,2


#### TODO
##### Features:
- <s>Died from ganks in laning phase pre 15 min (check ID from enemy Jungler)</s>

- Died with ... Gold

- <s>count 1v1 kill/deaths</s>  
  
- <s>Good roaming (assists pre 15 min)</s>

- <s>Warding (check ward placements pre 15 min) maybe in combination with ganks</s>

- CS per min (if data available)

- <s>KDA (kills and assists divided by deaths) als Output KDA String aus participantDf</s>

- <s>Tower Kills/Assists</s> 

- <s>Objective Kills/Assists (Baron, Dragon, Herald)</s> 



##### Features on the tableau visualization:

- <s>Timeline</s>

- <s>Champion Icons</s>

- <s>Damage Dealt Bar Chart (participantDf)</s>

- Game win (participantDf)



Check 1v1

In [49]:
kelodyEventsDf[kelodyEventsDf["kelodyAssist"] == False]

def check1v1(row, name):
    if not row["assistingParticipantIds"] and (row["victimName"] == name or row["killerName"] == name) and (row["type"] == "CHAMPION_KILL"):
        return True
    return False

kelodyEventsDf["1v1"] = kelodyEventsDf.apply(lambda row: check1v1(row, "Kelody"), axis = 1)
kelodyEventsDf[kelodyEventsDf["1v1"] == True]

,matchId,type,participantId,itemId,timestamp,level,wardType,creatorId,killerId,victimId,...,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist,1v1
5,EUW1_5829524931,CHAMPION_KILL,NaN,NaN,161459,NaN,NaN,NaN,8.0,3.0,...,NaN,3.0,NaN,NaN,Kelody,Diana,Christopheres,Katarina,False,True
20,EUW1_5829524931,CHAMPION_KILL,NaN,NaN,425348,NaN,NaN,NaN,8.0,2.0,...,NaN,7.0,NaN,NaN,Kelody,Diana,stoeckchen96,Kayn,False,True
59,EUW1_5829524931,CHAMPION_KILL,NaN,NaN,1212764,NaN,NaN,NaN,8.0,4.0,...,NaN,20.0,NaN,NaN,Kelody,Diana,iceblooded,Aphelios,False,True
67,EUW1_5829524931,CHAMPION_KILL,NaN,NaN,1366568,NaN,NaN,NaN,8.0,4.0,...,NaN,23.0,NaN,NaN,Kelody,Diana,iceblooded,Aphelios,False,True
75,EUW1_5829524931,CHAMPION_KILL,NaN,NaN,1535799,NaN,NaN,NaN,8.0,2.0,...,NaN,26.0,NaN,NaN,Kelody,Diana,stoeckchen96,Kayn,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,EUW1_5825747550,CHAMPION_KILL,NaN,NaN,556279,NaN,NaN,NaN,3.0,8.0,...,NaN,9.0,NaN,NaN,Kelody,Kassadin,Helium Submarine,Malzahar,False,True
1286,EUW1_5825747550,CHAMPION_KILL,NaN,NaN,833216,NaN,NaN,NaN,3.0,6.0,...,NaN,14.0,NaN,NaN,Kelody,Kassadin,From Unit 731,Camille,False,True
1292,EUW1_5825747550,CHAMPION_KILL,NaN,NaN,957080,NaN,NaN,NaN,3.0,8.0,...,NaN,16.0,NaN,NaN,Kelody,Kassadin,Helium Submarine,Malzahar,False,True
1293,EUW1_5825747550,CHAMPION_KILL,NaN,NaN,980475,NaN,NaN,NaN,3.0,10.0,...,NaN,16.0,NaN,NaN,Kelody,Kassadin,Big Moan,Teemo,False,True


In [50]:
def count1v1(row, name, killer):
  if killer:
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["1v1"] == True) & (kelodyEventsDf["killerName"] == name)])
  else: 
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["1v1"] == True) & (kelodyEventsDf["victimName"] == name)])

kelody["1v1Won"] = kelody.apply(lambda row: count1v1(row, "Kelody", True), axis=1)
kelody["1v1Lost"] = kelody.apply(lambda row: count1v1(row, "Kelody", False), axis=1)

In [51]:
def countTurretKills(row, name):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["type"] == "BUILDING_KILL") & (kelodyEventsDf["killerName"] == name) & (kelodyEventsDf["1v1"] == False)])

kelody["turretKills"] = kelody.apply(lambda row: countTurretKills(row, "Kelody"), axis=1)

In [52]:
def counEliteMonsterKills(row, monsterName):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["type"] == "ELITE_MONSTER_KILL") & (kelodyEventsDf["monsterType"] == monsterName)])

kelody["dragonKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "DRAGON"), axis=1)
kelody["baronKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "BARON_NASHOR"), axis=1)
kelody["heraldKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "RIFTHERALD"), axis=1)

In [53]:
def deathToGank(row, name):
    if row["1v1"] == False and row["type"] == "CHAMPION_KILL" and row["victimName"] == name: 
        enemyJungler = participantsDf.loc[(row["matchId"] == participantsDf["matchId"]) & (participantsDf["teamPosition"] == "JUNGLE")]["summonerName"].values[0]
        if row["killerName"] == enemyJungler:
            return True
        assistingParticipants = row["assistingParticipantIds"]
        for i in range(len(assistingParticipants)):
            if participantsDf.loc[(participantsDf["participant"] == assistingParticipants[i]) & (participantsDf["matchId"] == row["matchId"])]["teamPosition"].values[0] == "JUNGLE":
                return True
    return False
kelodyEventsDf["deathToGank"] = kelodyEventsDf.apply(lambda row: deathToGank(row, "Kelody"), axis=1)

In [54]:
def countDeathsToGanks(row, name):
    return len(
        kelodyEventsDf[
            (kelodyEventsDf["matchId"] == row["matchId"])
            & (kelodyEventsDf["deathToGank"] == True)
            & (kelodyEventsDf["victimName"] == name)
            & (kelodyEventsDf["timestampMIN"] <= 15)
        ]
    )


kelody["deathsToGanks"] = kelody.apply(
    lambda row: countDeathsToGanks(row, "Kelody"), axis=1
)


In [55]:
def countWardsPlaced(row, name):
    participantId = participantsDf.loc[(participantsDf["matchId"] == row["matchId"]) & (participantsDf["summonerName"] == name)]["participant"].values[0]
    return len(
        kelodyEventsDf[
            (kelodyEventsDf["matchId"] == row["matchId"])
            & (kelodyEventsDf["creatorId"] == participantId)
            & (kelodyEventsDf["type"] == "WARD_PLACED")
        ]
    )


kelody["wardsPlaced"] = kelody.apply(
    lambda row: countWardsPlaced(row, "Kelody"), axis=1
)


In [56]:
def countWardsPlacedBeforeGanked(row, name):
    participantId = participantsDf.loc[(participantsDf["matchId"] == row["matchId"]) & (
        participantsDf["summonerName"] == name)]["participant"].values[0]
    wardsPlacedTimestamps = kelodyEventsDf.loc[(kelodyEventsDf["matchId"] == row["matchId"]) & (
        kelodyEventsDf["creatorId"] == participantId) & (kelodyEventsDf["type"] == "WARD_PLACED")]["timestampMIN"].values
    ganksTimestamps = kelodyEventsDf.loc[
        (kelodyEventsDf["matchId"] == row["matchId"])
        & (kelodyEventsDf["deathToGank"] == True)
        & (kelodyEventsDf["victimName"] == name)
        & (kelodyEventsDf["timestampMIN"] <= 15)
    ]["timestampMIN"].values
    wardsPlacedBeforeGank = 0
    for timestamp in ganksTimestamps:
        for wardTimestamp in wardsPlacedTimestamps:
            if(timestamp - wardTimestamp <= 2 and timestamp - wardTimestamp > 0):
                wardsPlacedBeforeGank += 1
    return wardsPlacedBeforeGank

In [57]:
kelody["wardsPlacedBeforeGanks"] = kelody.apply(lambda row: countWardsPlacedBeforeGanked(row, "Kelody"), axis=1)

In [58]:
def checkRoamKills(row, name):
    if row["assistingParticipantIds"] and (row["killerName"] == name) and (row["type"] == "CHAMPION_KILL") and (row["timestampMIN"] <= 15):
        return True
    return False
-
def countRoamKills(row):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & ((kelodyEventsDf["kelodyAssist"] == True) & (kelodyEventsDf["timestampMIN"] <= 15) | (kelodyEventsDf["kelodyRoamKill"] == True))])

SyntaxError: invalid syntax (2034829799.py, line 5)

In [ ]:
kelodyEventsDf["kelodyRoamKill"] = kelodyEventsDf.apply(lambda row: checkRoamKills(row, "Kelody"), axis = 1)
kelody["roamKillsAssists"] = kelody.apply(lambda row: countRoamKills(row), axis=1)

In [ ]:
def gameWin(row, name):
    return participantsDf.loc[(participantsDf["matchId"] == row["matchId"]) & (participantsDf["summonerName"] == name)]["win"].values[0]

In [ ]:
kelody["gameWin"] = kelody.apply(lambda row: gameWin(row, "Kelody"), axis=1)

In [ ]:
def getKDA(row, name):
    kdaString = participantsDf.loc[(participantsDf["matchId"] == row["matchId"]) & (participantsDf["summonerName"] == name)]["kda"].values[0]
    kdaStringArr = kdaString.split("/")
    kdaArr = [int(kdaArrPart) for kdaArrPart in kdaStringArr]
    try:
        kda = (kdaArr[0]+kdaArr[2]) / kdaArr[1]
    except:
        kda = (kdaArr[0]+kdaArr[2]) / 1
    return round(kda, 2)

In [ ]:
kelody["kda"] = kelody.apply(lambda row: getKDA(row, "Kelody"), axis=1)

In [ ]:
kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost,turretKills,dragonKills,baronKills,heraldKills,deathsToGanks,wardsPlaced,wardsPlacedBeforeGanks,roamKillsAssists,gameWin,kda
7,EUW1_5829524931,8,1,5,1,1,0,0,0,4,14,4,10,False,1.90
12,EUW1_5829408711,3,1,5,3,0,0,0,0,1,13,0,6,False,1.17
27,EUW1_5829375821,8,0,1,0,1,0,0,1,0,4,0,1,True,1.00
36,EUW1_5827639869,7,2,1,1,0,1,0,0,2,3,1,4,False,1.20
46,EUW1_5827587343,7,2,1,0,0,1,0,0,3,5,2,9,False,1.67
58,EUW1_5827621876,9,3,1,0,1,0,0,0,2,13,2,3,False,1.88
67,EUW1_5827066877,8,4,0,8,0,2,1,0,0,14,0,0,False,1.25
77,EUW1_5827020811,8,2,6,6,0,0,0,0,2,19,0,0,False,0.78
88,EUW1_5826907936,9,4,1,3,0,0,0,0,1,8,2,6,False,1.10
93,EUW1_5826932804,4,2,2,0,4,0,1,0,1,16,0,9,True,2.45


In [ ]:
participantsDf.to_json('participants.json', orient='records')
matchesDf.to_json('matches.json', orient='records')
kelodyEventsDf.to_json('kelodyEvents.json', orient='records')
kelody.to_json('kelodyStats.json', orient='records')
matchTimelineDf.to_json('matchTimeline.json', orient='records')
